In [ ]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from pyspark.sql import SQLContext
from awsglue.context import GlueContext
from awsglue.job import Job

from pyspark.sql.types import *
from pyspark.sql.functions import pandas_udf, PandasUDFType,udf
from pyspark.sql.types import IntegerType, StringType
from pyspark.sql.functions import *

glueContext = GlueContext(SparkContext.getOrCreate())

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 0.35 
Authenticating with environment variables and user-defined glue_role_arn: arn:aws:iam::087763889191:role/service-role/AWSGlueServiceRole-imba-nining
Trying to create a Glue session for the kernel.
Worker Type: G.1X
Number of Workers: 5
Session ID: 01921107-44f4-481d-87fc-9d3e8f1284cc
Applying the following default arguments:
--glue_kernel_version 0.35
--enable-glue-datacatalog true
Waiting for session 01921107-44f4-481d-87fc-9d3e8f1284cc to get into ready status...
Session 01921107-44f4-481d-87fc-9d3e8f1284cc has been created




In [16]:
#crawler the data from data source, then define raw data zone as database for the data.
data_order = glueContext.create_dynamic_frame.from_catalog(database="raw data zone", table_name="orders")
print("Count: " + str(data_order.count()))
print(type(data_order))
data_order.printSchema()

print(type(data_order))

# data_product = glueContext.create_dynamic_frame.from_catalog(database="raw data zone", table_name="products")
# print("Count: " + str(data_product.count()))
# data_product.printSchema()

data_order_prior = glueContext.create_dynamic_frame.from_catalog(database="raw data zone", table_name="order_products")
print("Count: " + str(data_order_prior.count()))
data_order_prior.printSchema()

Count: 39
<class 'awsglue.dynamicframe.DynamicFrame'>
root
|-- order_id: long
|-- user_id: long
|-- eval_set: string
|-- order_number: long
|-- order_dow: long
|-- order_hour_of_day: long
|-- days_since_prior_order: long

<class 'awsglue.dynamicframe.DynamicFrame'>
Count: 49688
root
|-- product_id: long
|-- product_name: string
|-- aisle_id: long
|-- department_id: long

Count: 384
root
|-- order_id: long
|-- product_id: long
|-- add_to_cart_order: long
|-- reordered: long
|-- partition_0: string


In [17]:
#departments is just one table in database, so you can load other tables like aisles, orders, etc. one by one
data_order = data_order.toDF()
data_order.show(5)

# data_product = data_product.toDF()
# data_product.show(5)

data_order_prior =data_order_prior.toDF()
data_order_prior.show(5)

+--------+-------+--------+------------+---------+-----------------+----------------------+
|order_id|user_id|eval_set|order_number|order_dow|order_hour_of_day|days_since_prior_order|
+--------+-------+--------+------------+---------+-----------------+----------------------+
| 2539329|      1|   prior|           1|        2|                8|                  null|
| 2398795|      1|   prior|           2|        3|                7|                    15|
|  473747|      1|   prior|           3|        3|               12|                    21|
| 2254736|      1|   prior|           4|        4|                7|                    29|
|  431534|      1|   prior|           5|        4|               15|                    28|
+--------+-------+--------+------------+---------+-----------------+----------------------+
only showing top 5 rows

+--------+----------+-----------------+---------+--------------------+
|order_id|product_id|add_to_cart_order|reordered|         partition_0|
+----

In [20]:
data_order.createOrReplaceTempView("orders")
data_order_prior.createOrReplaceTempView("order_products")
print(type(data_order))
df_SQL = spark.sql("SELECT a.*,b.product_id, b.add_to_cart_order, b.reordered FROM orders as a JOIN order_products as b ON a.order_id = b.order_id WHERE a.eval_set = 'prior'")
df_SQL.show(5)

<class 'pyspark.sql.dataframe.DataFrame'>
+--------+-------+--------+------------+---------+-----------------+----------------------+----------+-----------------+---------+
|order_id|user_id|eval_set|order_number|order_dow|order_hour_of_day|days_since_prior_order|product_id|add_to_cart_order|reordered|
+--------+-------+--------+------------+---------+-----------------+----------------------+----------+-----------------+---------+
|  431534|      1|   prior|           5|        4|               15|                    28|       196|                1|        1|
|  431534|      1|   prior|           5|        4|               15|                    28|     12427|                2|        1|
|  431534|      1|   prior|           5|        4|               15|                    28|     10258|                3|        1|
|  431534|      1|   prior|           5|        4|               15|                    28|     25133|                4|        1|
|  431534|      1|   prior|           5| 

In [22]:
df_SQL.createOrReplaceTempView("order_products_prior")
user_features_1 = spark.sql("SELECT user_id, \
Max(order_number) AS user_orders, \
Sum(days_since_prior_order) AS user_period, \
Avg(days_since_prior_order) AS user_mean_days_since_prior \
FROM orders \
GROUP BY user_id")

user_features_1.show(5)

+-------+-----------+-----------+--------------------------+
|user_id|user_orders|user_period|user_mean_days_since_prior|
+-------+-----------+-----------+--------------------------+
|      2|         15|        228|        16.285714285714285|
|      3|         13|        144|                      12.0|
|      1|         11|        190|                      19.0|
+-------+-----------+-----------+--------------------------+


In [24]:
user_features_2 = spark.sql("SELECT user_id, \
Count(*) AS user_total_products, \
Count(DISTINCT product_id) AS user_distinct_products, \
Sum(CASE WHEN reordered = 1 THEN 1 ELSE 0 END) / Cast(Sum(CASE WHEN order_number > 1 THEN 1 ELSE 0 END) AS DOUBLE) AS user_reorder_ratio \
FROM order_products_prior  \
GROUP BY user_id")
user_features_2.show(5)


+-------+-------------------+----------------------+------------------+
|user_id|user_total_products|user_distinct_products|user_reorder_ratio|
+-------+-------------------+----------------------+------------------+
|      2|                195|                   102| 0.510989010989011|
|      3|                 88|                    33|0.7051282051282052|
|      1|                 59|                    18|0.7592592592592593|
+-------+-------------------+----------------------+------------------+


In [25]:
up_features = spark.sql("SELECT user_id, \
product_id, \
Count(*) AS up_orders, \
Min(order_number) AS up_first_order, \
Max(order_number) AS up_last_order, \
Avg(add_to_cart_order) AS up_average_cart_position \
FROM order_products_prior \
GROUP BY user_id, product_id") 
up_features.show(5)

+-------+----------+---------+--------------+-------------+------------------------+
|user_id|product_id|up_orders|up_first_order|up_last_order|up_average_cart_position|
+-------+----------+---------+--------------+-------------+------------------------+
|      2|     48110|        2|             1|            8|                     5.0|
|      3|     24810|        3|             1|           12|                     7.0|
|      2|     39928|        1|            14|           14|                    13.0|
|      3|     17668|        5|             1|           11|                     3.6|
|      2|     10305|        1|            12|           12|                    10.0|
+-------+----------+---------+--------------+-------------+------------------------+
only showing top 5 rows


In [26]:
prd_features_pri = spark.sql("SELECT *, Rank() OVER(partition BY user_id, product_id \
ORDER BY order_number) AS product_seq_time \
FROM order_products_prior")
prd_features_pri.show(5)

prd_features_pri.createOrReplaceTempView("prd_features_pri1")

prd_features = spark.sql("SELECT product_id, \
Count(*) AS prod_orders, \
Sum(reordered) AS prod_reorders, \
Sum(CASE WHEN product_seq_time = 1 THEN 1 ELSE 0 END) AS prod_first_orders, \
Sum(CASE WHEN product_seq_time = 2 THEN 1 ELSE 0 END) AS prod_second_orders \
FROM prd_features_pri1 \
GROUP BY product_id") 

prd_features.show(10)

+--------+-------+--------+------------+---------+-----------------+----------------------+----------+-----------------+---------+----------------+
|order_id|user_id|eval_set|order_number|order_dow|order_hour_of_day|days_since_prior_order|product_id|add_to_cart_order|reordered|product_seq_time|
+--------+-------+--------+------------+---------+-----------------+----------------------+----------+-----------------+---------+----------------+
| 2168274|      2|   prior|           1|        2|               11|                  null|     48110|                5|        0|               1|
|  788338|      2|   prior|           8|        1|               15|                    27|     48110|                5|        1|               2|
| 1374495|      3|   prior|           1|        1|               14|                  null|     24810|                9|        0|               1|
| 1972919|      3|   prior|           6|        0|               16|                     7|     24810|          

In [ ]:
from datetime import datetime, timezone, timedelta
import pytz
today = datetime.utcnow().replace(tzinfo=timezone.utc) 
Sydney = timezone(timedelta(hours=11))
date_time = today.astimezone(Sydney).strftime("%Y-%m-%d %H:%M:%S")
print(date_time)


user_features_1 =user_features_1.repartition(1)
user_features_1.write.parquet('s3://project-imba/data-lake/feature_extract/user_features_1'+'f'/{date_time}/')

In [30]:
# #data clean -- in this part, only one way, deduplication, is designed to demonstrate the clean process, you can add other methods to clean data
# loc = locals()      #get string name
# def get_variable_name(variable):
#     for k,v in loc.items():
#         if loc[k] is variable:
#             return k


# # def getString_name(df_name):
# #      return list(dict(df_name = df_name).keys())[0]#,type(list(dict(df_name = df_name).values())[0])

# def deduplicate_df(df_name):
#     if df_name.count() != df_name.distinct().count():
#         print( str(get_variable_name(df_name)) + ' have duplicated row:',df_name.count()- df_name.distinct().count())
#         df_name = df_name.dropDuplicates()
#         print('duplicated data has been cleared')
#     else:
#         print( str(get_variable_name(df_name)) + ' hasn\'t duplication, the totle distinct row',df_name.count())
#     return df_name

# df_orders_clean = deduplicate_df(df_orders)
# df_orders_clean.show(5)
# df_products_clean = deduplicate_df(df_products)
# df_order_products_clean = deduplicate_df(df_order_products)

NameError: name 'df_orders' is not defined


In [ ]:
# # put cleaned departments table into my datalake 
# df_products_clean =df_products_clean.repartition(1)
# df_products_clean.write.parquet('s3://project-imba/data-lake/raw-data/products_clean)

In [10]:
# wide_table_for_ML=(df_orders_clean.select("order_id","user_id","order_number")\
# .join(df_order_products,df_orders_clean.order_id==df_order_products.order_id,"left")\
# .drop(df_orders_clean.order_id)\
# .join(df_products,df_products.product_id == df_order_products.product_id,"left")\
# .drop(df_products.product_id)\
# .orderBy("user_id",df_orders_clean.order_number,"add_to_cart_order")\
# .drop("reordered","product_name"))\
# .select("user_id","order_id","order_number","department_id","aisle_id","product_id","add_to_cart_order","partition_0")\



In [12]:
# wide_table_for_ML.show(503)

+-------+--------+------------+-------------+--------+----------+-----------------+--------------------+
|user_id|order_id|order_number|department_id|aisle_id|product_id|add_to_cart_order|         partition_0|
+-------+--------+------------+-------------+--------+----------+-----------------+--------------------+
|      1| 2539329|           1|            7|      77|       196|                1|order_products_prior|
|      1| 2539329|           1|           16|      91|     14084|                2|order_products_prior|
|      1| 2539329|           1|           19|      23|     12427|                3|order_products_prior|
|      1| 2539329|           1|           19|      23|     26088|                4|order_products_prior|
|      1| 2539329|           1|           17|      54|     26405|                5|order_products_prior|
|      1| 2398795|           2|            7|      77|       196|                1|order_products_prior|
|      1| 2398795|           2|           19|     117| 